In [45]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import time
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor


In [44]:
# Data Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
 
# Machine Learning Models
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
 
# Model Evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, GridSearchCV
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
 
from sklearn.metrics import classification_report, accuracy_score
import random
random.seed(100)

In [3]:
os.chdir('D:/workspace/git_projects/Medicare-Claim-Fraud-Detection/data/interim')

In [5]:
data = pd.read_csv('training_data.csv')

## Issues
- outliers in **InscClaimAmtReimbursed IPAnnualReimbursementAmt IPAnnualDeductibleAmt OPAnnualReimbursementAmt OPAnnualDeductibleAmt**

In [8]:
data.isna().sum()

BeneID                             0
ClaimID                            0
Provider                           0
InscClaimAmtReimbursed             0
DeductibleAmtPaid                  0
ClaimPeriod                        0
TimeInHptal                        0
Diagnosis Count                    0
Procedures Count                   0
SamePhysician                      0
OPD_Flag                           0
PotentialFraud                     0
DOB                                0
DOD                                0
Gender                             0
Race                               0
RenalDiseaseIndicator              0
State                              0
County                             0
NoOfMonths_PartACov                0
NoOfMonths_PartBCov                0
ChronicCond_Alzheimer              0
ChronicCond_Heartfailure           0
ChronicCond_KidneyDisease          0
ChronicCond_Cancer                 0
ChronicCond_ObstrPulmonary         0
ChronicCond_Depression             0
C

In [7]:
del data['Unnamed: 0']

In [9]:
data.head()

,BeneID,ClaimID,Provider,InscClaimAmtReimbursed,DeductibleAmtPaid,ClaimPeriod,TimeInHptal,Diagnosis Count,Procedures Count,SamePhysician,...,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,BirthYear,Age,Alive,ChronicDisease_Count
0,BENE11001,CLM46614,PRV55912,26000,1068.0,6,6,9.0,0.0,0.0,...,Yes,Yes,36000,3204,60,70,1943,67.0,0,7.0
1,BENE11001,CLM66048,PRV55907,5000,1068.0,2,2,3.0,1.0,1.0,...,Yes,Yes,36000,3204,60,70,1943,67.0,0,7.0
2,BENE11001,CLM68358,PRV56046,5000,1068.0,3,3,6.0,0.0,0.0,...,Yes,Yes,36000,3204,60,70,1943,67.0,0,7.0
3,BENE11011,CLM38412,PRV52405,5000,1068.0,8,8,9.0,1.0,0.0,...,Yes,Yes,5000,1068,250,320,1914,96.0,0,6.0
4,BENE11014,CLM63689,PRV56614,10000,1068.0,17,17,9.0,1.0,0.0,...,No,No,21260,2136,120,100,1938,72.0,0,5.0


In [20]:
data.describe()

,InscClaimAmtReimbursed,DeductibleAmtPaid,ClaimPeriod,TimeInHptal,Diagnosis Count,Procedures Count,PotentialFraud,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,BirthYear,Age,Alive,ChronicDisease_Count
count,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.00000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.0,558211.000000
mean,997.012133,78.294788,1.727940,0.410762,3.010897,0.053557,0.381211,378.588195,11.931472,11.93877,5227.971466,568.756807,2278.225348,649.698745,1935.723180,73.769770,0.0,4.498616
std,3821.534891,273.814128,4.904984,2.112693,2.448213,0.280534,0.485685,265.215531,0.889712,0.78590,11786.274732,1179.172616,3881.846386,1002.020811,13.011761,13.022524,0.0,2.332301
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,-8000.000000,0.000000,-70.000000,0.000000,1909.000000,26.000000,0.0,0.000000
25%,40.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,150.000000,12.000000,12.00000,0.000000,0.000000,460.000000,120.000000,1927.000000,68.000000,0.0,3.000000
50%,80.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,350.000000,12.000000,12.00000,0.000000,0.000000,1170.000000,340.000000,1935.000000,75.000000,0.0,5.000000
75%,300.000000,0.000000,0.000000,0.000000,4.000000,0.000000,1.000000,570.000000,12.000000,12.00000,6000.000000,1068.000000,2590.000000,790.000000,1941.000000,82.000000,0.0,6.000000
max,125000.000000,1068.000000,36.000000,35.000000,10.000000,5.000000,1.000000,999.000000,12.000000,12.00000,161470.000000,38272.000000,102960.000000,13840.000000,1983.000000,101.000000,0.0,11.000000


In [17]:
data[data['DeductibleAmtPaid'] == -1].shape

(0, 40)

In [15]:
#replaced 899 values that are -1 with 0 
data['DeductibleAmtPaid'] =  data['DeductibleAmtPaid'].replace(-1,0)

In [19]:
#convert to categorical datatype
data['SamePhysician'] = data['SamePhysician'].astype('object')
data['OPD_Flag'] = data['OPD_Flag'].astype('object')
data['RenalDiseaseIndicator'] = data['RenalDiseaseIndicator'].astype('object')

In [30]:
data.shape

(558211, 40)

In [28]:
data[(data['IPAnnualReimbursementAmt'] <= 0) & (data['IPAnnualDeductibleAmt'] <= 0)].shape

(369243, 40)

In [29]:
data[(data['OPAnnualDeductibleAmt'] < 0) & (data['OPAnnualReimbursementAmt'] < 0)].shape


(0, 40)

In [34]:
data = data[data['IPAnnualReimbursementAmt'] >=0].reset_index(drop=True)

In [33]:
data.shape

(558154, 40)

In [35]:
data = data[data['OPAnnualReimbursementAmt'] >= 0].reset_index(drop=True)

In [36]:
data.shape


(558138, 40)

In [ ]:
data.head(100)

In [38]:
data.describe() 

,InscClaimAmtReimbursed,DeductibleAmtPaid,ClaimPeriod,TimeInHptal,Diagnosis Count,Procedures Count,PotentialFraud,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,BirthYear,Age,Alive,ChronicDisease_Count
count,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.0,558138.000000
mean,996.806184,78.275842,1.727711,0.410630,3.010793,0.053537,0.381212,378.587666,11.931463,11.938762,5228.095847,568.674654,2278.394824,649.732926,1935.723183,73.769772,0.0,4.498626
std,3820.614730,273.782870,4.904651,2.112183,2.448085,0.280484,0.485685,265.210830,0.889769,0.785951,11785.791749,1179.216729,3882.045712,1002.070824,13.011398,13.022134,0.0,2.332332
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1909.000000,26.000000,0.0,0.000000
25%,40.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,150.000000,12.000000,12.000000,0.000000,0.000000,460.000000,120.000000,1927.000000,68.000000,0.0,3.000000
50%,80.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,350.000000,12.000000,12.000000,0.000000,0.000000,1170.000000,340.000000,1935.000000,75.000000,0.0,5.000000
75%,300.000000,0.000000,0.000000,0.000000,4.000000,0.000000,1.000000,570.000000,12.000000,12.000000,6000.000000,1068.000000,2590.000000,790.000000,1941.000000,82.000000,0.0,6.000000
max,125000.000000,1068.000000,36.000000,35.000000,10.000000,5.000000,1.000000,999.000000,12.000000,12.000000,161470.000000,38272.000000,102960.000000,13840.000000,1983.000000,101.000000,0.0,11.000000


In [ ]:
data[data['InscClaimAmtReimbursed'] > 100000].head(100)

In [49]:
data[data['DeductibleAmtPaid']>0].shape

(60592, 40)

## Treating Outliers

**InscClaimAmtReimbursed IPAnnualReimbursementAmt IPAnnualDeductibleAmt OPAnnualReimbursementAmt OPAnnualDeductibleAmt**

In [51]:
def treat_outliers(df, columns):
   
    for column in columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        print('lower', lower_bound)

        upper_bound = Q3 + 1.5 * IQR
        print('upper', upper_bound)
        # Cap outliers
        df[column] = np.where(df[column] < lower_bound, lower_bound, df[column])
        df[column] = np.where(df[column] > upper_bound, upper_bound, df[column])
    return df
 

In [53]:
columns = ['InscClaimAmtReimbursed', 'IPAnnualReimbursementAmt', 'IPAnnualDeductibleAmt', 'OPAnnualReimbursementAmt', 'OPAnnualDeductibleAmt']
data = treat_outliers(data, columns)

lower -350.0
upper 690.0
lower -9000.0
upper 15000.0
lower -1602.0
upper 2670.0
lower -2735.0
upper 5785.0
lower -885.0
upper 1795.0


In [54]:
data.describe()

,InscClaimAmtReimbursed,DeductibleAmtPaid,ClaimPeriod,TimeInHptal,Diagnosis Count,Procedures Count,PotentialFraud,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,BirthYear,Age,Alive,ChronicDisease_Count
count,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.0,558138.000000
mean,214.277258,78.275842,1.727711,0.410630,3.010793,0.053537,0.381212,378.587666,11.931463,11.938762,3346.451523,502.191917,1775.557649,533.814659,1935.723183,73.769772,0.0,4.498626
std,248.572190,273.782870,4.904651,2.112183,2.448085,0.280484,0.485685,265.210830,0.889769,0.785951,5434.170231,785.740737,1692.759775,526.272005,13.011398,13.022134,0.0,2.332332
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1909.000000,26.000000,0.0,0.000000
25%,40.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,150.000000,12.000000,12.000000,0.000000,0.000000,460.000000,120.000000,1927.000000,68.000000,0.0,3.000000
50%,80.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,350.000000,12.000000,12.000000,0.000000,0.000000,1170.000000,340.000000,1935.000000,75.000000,0.0,5.000000
75%,300.000000,0.000000,0.000000,0.000000,4.000000,0.000000,1.000000,570.000000,12.000000,12.000000,6000.000000,1068.000000,2590.000000,790.000000,1941.000000,82.000000,0.0,6.000000
max,690.000000,1068.000000,36.000000,35.000000,10.000000,5.000000,1.000000,999.000000,12.000000,12.000000,15000.000000,2670.000000,5785.000000,1795.000000,1983.000000,101.000000,0.0,11.000000


In [58]:
data.drop(columns={'DOB', 'DOD', 'Alive', 'State', 'County'},inplace=True)

In [74]:
data.columns

Index(['ClaimID', 'Provider', 'InscClaimAmtReimbursed', 'DeductibleAmtPaid',
       'ClaimPeriod', 'TimeInHptal', 'Diagnosis Count', 'Procedures Count',
       'SamePhysician', 'OPD_Flag', 'PotentialFraud', 'Gender', 'Race',
       'RenalDiseaseIndicator', 'NoOfMonths_PartACov', 'NoOfMonths_PartBCov',
       'ChronicCond_Alzheimer', 'ChronicCond_Heartfailure',
       'ChronicCond_KidneyDisease', 'ChronicCond_Cancer',
       'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression',
       'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart',
       'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',
       'ChronicCond_stroke', 'IPAnnualReimbursementAmt',
       'IPAnnualDeductibleAmt', 'OPAnnualReimbursementAmt',
       'OPAnnualDeductibleAmt', 'BirthYear', 'Age', 'ChronicDisease_Count'],
      dtype='object')

## Categorical Columns

In [65]:
data.select_dtypes(include='object')

,BeneID,ClaimID,Provider,SamePhysician,OPD_Flag,Gender,Race,RenalDiseaseIndicator,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke
0,BENE11001,CLM46614,PRV55912,0.0,0,Male,White,0,Yes,No,Yes,No,No,Yes,Yes,Yes,No,Yes,Yes
1,BENE11001,CLM66048,PRV55907,1.0,0,Male,White,0,Yes,No,Yes,No,No,Yes,Yes,Yes,No,Yes,Yes
2,BENE11001,CLM68358,PRV56046,0.0,0,Male,White,0,Yes,No,Yes,No,No,Yes,Yes,Yes,No,Yes,Yes
3,BENE11011,CLM38412,PRV52405,0.0,0,Female,Black or African American,0,No,Yes,Yes,No,No,Yes,Yes,No,No,Yes,Yes
4,BENE11014,CLM63689,PRV56614,0.0,0,Female,White,1,No,Yes,Yes,No,Yes,Yes,No,Yes,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558133,BENE159198,CLM510792,PRV53699,1.0,1,Female,White,0,Yes,Yes,No,No,No,Yes,Yes,No,No,Yes,No
558134,BENE159198,CLM551294,PRV53702,0.0,1,Female,White,0,Yes,Yes,No,No,No,Yes,Yes,No,No,Yes,No
558135,BENE159198,CLM596444,PRV53676,0.0,1,Female,White,0,Yes,Yes,No,No,No,Yes,Yes,No,No,Yes,No
558136,BENE159198,CLM636992,PRV53689,0.0,1,Female,White,0,Yes,Yes,No,No,No,Yes,Yes,No,No,Yes,No


In [66]:
data['Gender'].value_counts()

Gender
Female    323072
Male      235066
Name: count, dtype: int64

In [67]:
data['Race'].value_counts()

Race
White                        470982
Black or African American     55628
Other                         19708
Hispanic                      11820
Name: count, dtype: int64

In [68]:
data['RenalDiseaseIndicator'].value_counts()

RenalDiseaseIndicator
0    448294
1    109844
Name: count, dtype: int64

In [69]:
data['PotentialFraud'].value_counts()

PotentialFraud
0    345369
1    212769
Name: count, dtype: int64

## Model Building

### Base Model

In [71]:
data['ClaimID'].nunique()

558138

In [72]:
del data['BeneID']

In [73]:
data.shape

(558138, 34)

#### One Hot Encoding

In [75]:
cat_cols = data.select_dtypes(include='object')

In [81]:
cat_cols = ['SamePhysician', 'OPD_Flag', 'Gender', 'Race',
       'RenalDiseaseIndicator', 'ChronicCond_Alzheimer',
       'ChronicCond_Heartfailure', 'ChronicCond_KidneyDisease',
       'ChronicCond_Cancer', 'ChronicCond_ObstrPulmonary',
       'ChronicCond_Depression', 'ChronicCond_Diabetes',
       'ChronicCond_IschemicHeart', 'ChronicCond_Osteoporasis',
       'ChronicCond_rheumatoidarthritis', 'ChronicCond_stroke']

In [83]:
cat_cols


['SamePhysician',
 'OPD_Flag',
 'Gender',
 'Race',
 'RenalDiseaseIndicator',
 'ChronicCond_Alzheimer',
 'ChronicCond_Heartfailure',
 'ChronicCond_KidneyDisease',
 'ChronicCond_Cancer',
 'ChronicCond_ObstrPulmonary',
 'ChronicCond_Depression',
 'ChronicCond_Diabetes',
 'ChronicCond_IschemicHeart',
 'ChronicCond_Osteoporasis',
 'ChronicCond_rheumatoidarthritis',
 'ChronicCond_stroke']

In [78]:
data.shape

(558138, 34)

In [85]:
pd.get_dummies(data[cat_cols],drop_first=True)

,SamePhysician_1.0,OPD_Flag_1,Gender_Male,Race_Hispanic,Race_Other,Race_White,RenalDiseaseIndicator_1,ChronicCond_Alzheimer_Yes,ChronicCond_Heartfailure_Yes,ChronicCond_KidneyDisease_Yes,ChronicCond_Cancer_Yes,ChronicCond_ObstrPulmonary_Yes,ChronicCond_Depression_Yes,ChronicCond_Diabetes_Yes,ChronicCond_IschemicHeart_Yes,ChronicCond_Osteoporasis_Yes,ChronicCond_rheumatoidarthritis_Yes,ChronicCond_stroke_Yes
0,False,False,True,False,False,True,False,True,False,True,False,False,True,True,True,False,True,True
1,True,False,True,False,False,True,False,True,False,True,False,False,True,True,True,False,True,True
2,False,False,True,False,False,True,False,True,False,True,False,False,True,True,True,False,True,True
3,False,False,False,False,False,False,False,False,True,True,False,False,True,True,False,False,True,True
4,False,False,False,False,False,True,True,False,True,True,False,True,True,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558133,True,True,False,False,False,True,False,True,True,False,False,False,True,True,False,False,True,False
558134,False,True,False,False,False,True,False,True,True,False,False,False,True,True,False,False,True,False
558135,False,True,False,False,False,True,False,True,True,False,False,False,True,True,False,False,True,False
558136,False,True,False,False,False,True,False,True,True,False,False,False,True,True,False,False,True,False


In [ ]:
data['RenalDiseaseIndicator'] = data['RenalDiseaseIndicator'].replace({0:'No', 1:'Yes'})
data['OPD_Flag'] = data['OPD_Flag'].replace({0:'No', 1:'Yes'})
data['SamePhysician'] = data['SamePhysician'].astype(int).replace({0:'No', 1:'Yes'})